# Goal: Image to 3D model (PIFuHD)

**Zhengxiao Wu, 2020213235**

References: [PIFuHD: Multi-Level Pixel-Aligned Implicit Function for High-Resolution 3D Human Digitization](https://shunsukesaito.github.io/PIFuHD/)


## Before running:  

```
!git clone https://github.com/facebookresearch/pifuhd
!git clone https://github.com/Daniil-Osokin/lightweight-human-pose-estimation.pytorch.git

%cd /content/lightweight-human-pose-estimation.pytorch/
!wget https://download.01.org/opencv/openvino_training_extensions/models/human_pose_estimation/checkpoint_iter_370000.pth

%cd /content/pifuhd/
!sh ./scripts/download_trained_model.sh
```

Then install PyTorch.

In [1]:
image_path = '../istockphoto-1307214736-170667a.jpg'

%cd C:\Users\wzx14\big_dev\PIFuHD\lightweight-human-pose-estimation.pytorch

C:\Users\wzx14\big_dev\PIFuHD\lightweight-human-pose-estimation.pytorch


In [2]:

import torch
import cv2
import numpy as np
from models.with_mobilenet import PoseEstimationWithMobileNet
from modules.keypoints import extract_keypoints, group_keypoints
from modules.load_state import load_state
from modules.pose import Pose, track_poses
import demo
from IPython.display import clear_output

def get_rect(net, images, height_size):
    net = net.eval()

    stride = 8
    upsample_ratio = 4
    num_keypoints = Pose.num_kpts
    previous_poses = []
    delay = 33
    for image in images:
        rect_path = image.replace('.%s' % (image.split('.')[-1]), '_rect.txt')
        img = cv2.imread(image, cv2.IMREAD_COLOR)
        orig_img = img.copy()
        orig_img = img.copy()
        heatmaps, pafs, scale, pad = demo.infer_fast(net, img, height_size, stride, upsample_ratio, cpu=False)

        total_keypoints_num = 0
        all_keypoints_by_type = []
        for kpt_idx in range(num_keypoints):  # 19th for bg
            total_keypoints_num += extract_keypoints(heatmaps[:, :, kpt_idx], all_keypoints_by_type, total_keypoints_num)

        pose_entries, all_keypoints = group_keypoints(all_keypoints_by_type, pafs)
        for kpt_id in range(all_keypoints.shape[0]):
            all_keypoints[kpt_id, 0] = (all_keypoints[kpt_id, 0] * stride / upsample_ratio - pad[1]) / scale
            all_keypoints[kpt_id, 1] = (all_keypoints[kpt_id, 1] * stride / upsample_ratio - pad[0]) / scale
        current_poses = []

        rects = []
        for n in range(len(pose_entries)):
            if len(pose_entries[n]) == 0:
                continue
            pose_keypoints = np.ones((num_keypoints, 2), dtype=np.int32) * -1
            valid_keypoints = []
            for kpt_id in range(num_keypoints):
                if pose_entries[n][kpt_id] != -1.0:  # keypoint was found
                    pose_keypoints[kpt_id, 0] = int(all_keypoints[int(pose_entries[n][kpt_id]), 0])
                    pose_keypoints[kpt_id, 1] = int(all_keypoints[int(pose_entries[n][kpt_id]), 1])
                    valid_keypoints.append([pose_keypoints[kpt_id, 0], pose_keypoints[kpt_id, 1]])
            valid_keypoints = np.array(valid_keypoints)
            
            if pose_entries[n][10] != -1.0 or pose_entries[n][13] != -1.0:
              pmin = valid_keypoints.min(0)
              pmax = valid_keypoints.max(0)

              center = (0.5 * (pmax[:2] + pmin[:2])).astype(np.int)
              radius = int(0.65 * max(pmax[0]-pmin[0], pmax[1]-pmin[1]))
            elif pose_entries[n][10] == -1.0 and pose_entries[n][13] == -1.0 and pose_entries[n][8] != -1.0 and pose_entries[n][11] != -1.0:
              # if leg is missing, use pelvis to get cropping
              center = (0.5 * (pose_keypoints[8] + pose_keypoints[11])).astype(np.int)
              radius = int(1.45*np.sqrt(((center[None,:] - valid_keypoints)**2).sum(1)).max(0))
              center[1] += int(0.05*radius)
            else:
              center = np.array([img.shape[1]//2,img.shape[0]//2])
              radius = max(img.shape[1]//2,img.shape[0]//2)

            x1 = center[0] - radius
            y1 = center[1] - radius

            rects.append([x1, y1, 2*radius, 2*radius])

        np.savetxt(rect_path, np.array(rects), fmt='%d')

In [3]:
%cd C:\Users\wzx14\big_dev\PIFuHD\lightweight-human-pose-estimation.pytorch

import cv2
img = cv2.imread(image_path)
image_path = '../sample_images/Img.png'
cv2.imwrite(image_path, img)

import os
image_dir = os.path.dirname(image_path)
file_name = os.path.splitext(os.path.basename(image_path))[0]

obj_path='../results/pifuhd_final/recon/result_%s_256.obj' % file_name
out_img_path='../results/pifuhd_final/recon/result_%s_256.png' % file_name
video_path='../results/pifuhd_final/recon/result_%s_256.mp4' % file_name
video_display_path='../results/pifuhd_final/result_%s_256_display.mp4' % file_name

C:\Users\wzx14\big_dev\PIFuHD\lightweight-human-pose-estimation.pytorch


In [4]:
%cd C:\Users\wzx14\big_dev\PIFuHD\lightweight-human-pose-estimation.pytorch

net = PoseEstimationWithMobileNet()
checkpoint = torch.load('checkpoint_iter_370000.pth', map_location='cpu')
load_state(net, checkpoint)
get_rect(net.cuda(), [image_path], 512)

%cd C:\Users\wzx14\big_dev\PIFuHD\pifuhd
!python -m apps.simple_test -r 256 --use_rect -i $image_dir

# clear_output()

C:\Users\wzx14\big_dev\PIFuHD\lightweight-human-pose-estimation.pytorch
C:\Users\wzx14\big_dev\PIFuHD\pifuhd


C:\Users\wzx14\AppData\Local\Temp\ipykernel_60904\4097356389.py:58: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  center = (0.5 * (pose_keypoints[8] + pose_keypoints[11])).astype(np.int)


^C


In [ ]:
# %cd C:\Users\wzx14\big_dev\PIFuHD\pifuhd

# from lib.colab_util import generate_video_from_obj, set_renderer, video

# renderer = set_renderer()

# obj_path=r'C:\Users\wzx14\big_dev\PIFuHD\pifuhd\results\pifuhd_final/recon/result_%s_256.obj' % file_name
# generate_video_from_obj(obj_path, out_img_path, video_path, renderer)

# !ffmpeg -i $video_path -vcodec libx264 $video_display_path -y -loglevel quiet